In [7]:
import pandas as pd
import geopandas as gpd
import numpy as np
import itertools

import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
import seaborn as sns

import scipy.stats as stats
#from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score


In [8]:
acc = pd.read_csv('../data/cleaned/coll_coord.csv', low_memory=False)

In [9]:
# acc['latitude'][0]

In [24]:
acc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202130 entries, 0 to 202129
Data columns (total 41 columns):
Unnamed: 0         202130 non-null int64
OBJECTID           202130 non-null int64
INCKEY             202130 non-null int64
COLDETKEY          202130 non-null int64
REPORTNO           202130 non-null object
STATUS             202130 non-null object
ADDRTYPE           202130 non-null object
INTKEY             67263 non-null float64
LOCATION           202130 non-null object
EXCEPTRSNCODE      84141 non-null object
EXCEPTRSNDESC      7190 non-null object
SEVERITYCODE       202129 non-null object
SEVERITYDESC       202130 non-null object
COLLISIONTYPE      179243 non-null object
PERSONCOUNT        202130 non-null int64
PEDCOUNT           202130 non-null int64
PEDCYLCOUNT        202130 non-null int64
VEHCOUNT           202130 non-null int64
INJURIES           202130 non-null int64
SERIOUSINJURIES    202130 non-null int64
FATALITIES         202130 non-null int64
INCDATE            20

NoneType

In [25]:
acc.columns

Index(['Unnamed: 0', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS',
       'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC',
       'SEVERITYCODE', 'SEVERITYDESC', 'COLLISIONTYPE', 'PERSONCOUNT',
       'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INJURIES', 'SERIOUSINJURIES',
       'FATALITIES', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE',
       'SDOT_COLDESC', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND',
       'LIGHTCOND', 'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE',
       'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR', 'longitude',
       'latitude'],
      dtype='object')

In [63]:
acc.head(1)

,Unnamed: 0,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,...,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,longitude,latitude
0,0,1,300647,301861,E828639,Matched,Block,NaN,WEEDIN E PL NE BETWEEN WEEDIN PL NE AND 8TH AV...,,...,NaN,NaN,NaN,10,Entering at angle,0,0,N,-122.319483,47.675997


In [17]:
acc.iloc[0]

Unnamed: 0                                                         0
OBJECTID                                                           1
INCKEY                                                        300647
COLDETKEY                                                     301861
REPORTNO                                                     E828639
STATUS                                                       Matched
ADDRTYPE                                                       Block
INTKEY                                                           NaN
LOCATION           WEEDIN E PL NE BETWEEN WEEDIN PL NE AND 8TH AV...
EXCEPTRSNCODE                                                       
EXCEPTRSNDESC                                                    NaN
SEVERITYCODE                                                       1
SEVERITYDESC                          Property Damage Only Collision
COLLISIONTYPE                                                 Angles
PERSONCOUNT                       

In [64]:
acc.SEGLANEKEY.unique()

array([    0, 42680, 45509, ...,  6397, 44360,  3800])

In [46]:
acc.LIGHTCOND.unique()

array(['Daylight', 'Unknown', 'Dark - Street Lights On', 'Dusk',
       'Dark - No Street Lights', nan, 'Other', 'Dawn',
       'Dark - Street Lights Off'], dtype=object)

In [55]:
acc.COLLISIONTYPE.unique()

array(['Angles', 'Rear Ended', 'Parked Car', 'Other', 'Sideswipe',
       'Right Turn', 'Pedestrian', 'Head On', 'Left Turn', nan, 'Cycles'],
      dtype=object)

In [58]:
len(acc['ST_COLDESC'].unique())

63

In [59]:
len(acc['ST_COLCODE'].unique())

64

In [48]:
acc.JUNCTIONTYPE.unique()

array(['Driveway Junction', 'Mid-Block (not related to intersection)',
       'At Intersection (intersection related)',
       'At Intersection (but not related to intersection)', nan,
       'Mid-Block (but intersection related)', 'Ramp Junction', 'Unknown'],
      dtype=object)

In [49]:
acc.SDOT_COLCODE.unique()

array([11., 14., 28.,  0., 13., 24., 16., 15., 51., 26., 18., 54., 12.,
       48., 29., 64., 25., 69., 31., 21., 27., 23., 34., 55., 56., 33.,
       nan, 44., 32., 66., 36., 22., 47., 58., 35., 61., 53., 46., 68.,
       52.])

In [53]:
len(acc.SDOT_COLDESC.unique())

40

In [51]:
acc.ROADCOND.unique()

array(['Dry', 'Unknown', 'Wet', nan, 'Other', 'Snow/Slush', 'Ice',
       'Standing Water', 'Sand/Mud/Dirt', 'Oil'], dtype=object)

In [54]:
acc.ST_COLCODE.unique()

array(['10', '14', '32', '50', '11', '16', '0', '12', '24', '28', ' ',
       '20', '6', '23', '13', '26', '5', '1', '56', '7', '27', '15', '2',
       '40', '52', '71', '72', '30', '51', '88', '84', '81', '73', '25',
       '29', '41', '48', nan, '45', '53', '19', '3', '74', '22', '21',
       '83', '4', '66', '57', '65', '82', '67', '49', '31', '42', '43',
       '64', '54', '60', '87', '17', '8', '18', '85'], dtype=object)

In [61]:
acc.columns

Index(['Unnamed: 0', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'STATUS',
       'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC',
       'SEVERITYCODE', 'SEVERITYDESC', 'COLLISIONTYPE', 'PERSONCOUNT',
       'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INJURIES', 'SERIOUSINJURIES',
       'FATALITIES', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE',
       'SDOT_COLDESC', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND',
       'LIGHTCOND', 'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE',
       'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR', 'longitude',
       'latitude'],
      dtype='object')

In [62]:
[(column, acc[column].unique()) for column in acc.columns if len(acc[column].unique()) < 100]

[('STATUS', array(['Matched', 'Unmatched'], dtype=object)),
 ('ADDRTYPE', array(['Block', 'Intersection'], dtype=object)),
 ('EXCEPTRSNCODE', array([' ', 'NEI', nan], dtype=object)),
 ('EXCEPTRSNDESC', array([nan,
         'Not Enough Information, or Insufficient Location Information'],
        dtype=object)),
 ('SEVERITYCODE', array(['1', '2', '2b', '0', '3', nan], dtype=object)),
 ('SEVERITYDESC', array(['Property Damage Only Collision', 'Injury Collision',
         'Serious Injury Collision', 'Unknown', 'Fatality Collision'],
        dtype=object)),
 ('COLLISIONTYPE',
  array(['Angles', 'Rear Ended', 'Parked Car', 'Other', 'Sideswipe',
         'Right Turn', 'Pedestrian', 'Head On', 'Left Turn', nan, 'Cycles'],
        dtype=object)),
 ('PERSONCOUNT',
  array([ 2,  3,  4,  7,  5,  1,  0,  6,  9,  8, 13, 12, 16, 17, 11, 26, 22,
         10, 37, 28, 36, 14, 19, 53, 30, 29, 23, 44, 15, 32, 21, 20, 41, 27,
         35, 43, 81, 18, 48, 25, 24, 34, 57, 39, 47, 31, 54, 93])),
 ('PEDCOUNT',